# Fuzzy Filtering on Metadata with KDB.AI Vector Database

##### Note: This example requires a KDB.AI server. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

#### In this example, we will show how to use metadata filtering along with fuzzy filtering in a KDB.AI vector database to increase the speed and accuracy of vector similarity searches.

#### NOTE! KDB.AI Also has 'fuzzy filter' capabilities on metadata columns.
Data often contains errors such as typos, misspellings, or international spelling variations, which can hinder the accuracy of search results. Fuzzy filters address this issue by enabling the retrieval of documents that contain terms and metadata entries similar to the specified query term and filters, even if there are slight variations.

There are many distance metrics you can use for fuzzy filtering, it defaults to Levenshtein distance, but you have the ability to choose the distance metric from a variety of options including: Levenshtein, Damerau-Levenshtein, Hamming, Indel, Jaro, JaroWinkler, Longest Common Subsequence, Optimal String Alignment (OSA), Prefix, or Postfix.

#### Agenda:
1. Set Up
2. Data Import and Understanding
3. Set Up KDB.AI Vector Database
4. Insert Movie Data into the KDB.AI table
5. Run Filtered & Fuzzy Similarity Searches on our KDB.AI vector database

Movie Dataset Source: https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots

## 1. Set Up
#### Installs, imports, and API Key setup

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

In [ ]:
!pip install kdbai_client
!pip install sentence_transformers

In [2]:
import pandas as pd
import os
from getpass import getpass

In [3]:
### !!! Only run this cell if you need to download the data into your environment, for example in Colab
### This downloads movie data
if os.path.exists("./data/filtered_embedded_movies.pkl") == False:
  !mkdir ./data
  !wget -P ./data https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/metadata_filtering/data/filtered_embedded_movies.pkl

## 2. Data Import and Understanding
### Import movies dataframe

In [4]:
# Read in the Movies dataframe
df = pd.read_pickle("./data/filtered_embedded_movies.pkl")

### Initial data exploration: Let's understand the data!

In [5]:
# How many rows do we have?
print(df.shape[0])

19161


In [6]:
# What columns do we have?
for column in df.columns:
    print(column)

ReleaseYear
Title
Origin
Director
Cast
Genre
Plot
embeddings


In [7]:
# Let us inspect the dataframe
df.head()

,ReleaseYear,Title,Origin,Director,Cast,Genre,Plot,embeddings
0,1975,The Candy Tangerine Man,American,Matt Cimber,John Daniels Eli Haines Tom Hankason,action,A successful Los Angeles-based businessperson ...,"[-0.06835174, -0.013138616, -0.12417501, 0.002..."
1,1975,Capone,American,Steve Carver,Ben Gazzara Susan Blakely John Cassavetes Sylv...,crime drama,The story is of the rise and fall of the Chica...,"[-0.01411798, 0.040705115, -0.0014280609, 0.00..."
2,1975,Cleopatra Jones and the Casino of Gold,American,Charles Bail,Tamara Dobson Stella Stevens,action,The story begins with two government agents Ma...,"[-0.0925895, 0.01188509, -0.08999529, -0.01541..."
3,1975,Conduct Unbecoming,American,Michael Anderson,Stacy Keach Richard Attenborough Christopher P...,drama,Around 1880 two young British officers arrive ...,"[-0.07435084, -0.06386179, 0.017042944, 0.0288..."
4,1975,Cooley High,American,Michael Schultz,Lawrence Hilton-Jacobs Glynn Turman Garrett Mo...,comedy,Set in 1964 Chicago Preach an aspiring playwri...,"[-0.041632336, 0.037923656, -0.072276264, -0.0..."


## 3. Set up KDB.AI Vector Database
Now that we understand our dataset, we can set up our vector db



In [8]:
# vector DB
import os
from getpass import getpass
import kdbai_client as kdbai
import time

### Connect to KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
#Set up KDB.AI server endpoint 
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else "http://localhost:8082"
)

#connect to KDB.AI Server, default mode is qipc
session = kdbai.Session(endpoint=KDBAI_ENDPOINT)


### Set up the table schema
Have a table column for each column in the dataframe, as well as an 'embeddings' column for the movie description embeddings

In [12]:
# Set up the schema for KDB.AI table, specifying column information
table_schema = [
        {"name": "ReleaseYear", "type": "int64"},
        {"name": "Title", "type": "str"},
        {"name": "Origin", "type": "str"},
        {"name": "Director", "type": "str"},
        {"name": "Cast", "type": "str"},
        {"name": "Genre", "type": "str"},
        {"name": "Plot", "type": "str"},
        {"name": "embeddings", "type": "float64s"},
    ]

# Set up the index with 384 dimensions, Euclidean Distance, and flat index
indexes = [
    {
        "name": "flat_index",
        "type": "flat",
        "column": "embeddings",
        "params": {"dims": 384, "metric": "L2"},
    }
]


### Create a table called "metadata_demo"
First check if the table already exists, then create a new table with the table schema from above

In [13]:
# get the database connection. Default database name is 'default'
database = session.database('default')

# First ensure the table does not already exist
try:
    database.table("metadata_demo").drop()
    time.sleep(5)
except kdbai.KDBAIException:
    pass

In [14]:
# Create the table called "metadata_demo"
table = database.create_table("metadata_demo", table_schema, indexes=indexes)

## 4. Insert Movie Data into the KDB.AI table

In [15]:
# Insert the data into the table, split into 2000 row batches
from tqdm import tqdm
n = 2000  # chunk row size

for i in tqdm(range(0, df.shape[0], n)):
    table.insert(df[i:i+n].reset_index(drop=True))

100%|██████████| 10/10 [00:01<00:00,  5.29it/s]


In [16]:
# function to view the dataframe within the table
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

In [17]:
# View contents of the table
show_df(table.query())

(19161, 8)


,ReleaseYear,Title,Origin,Director,Cast,Genre,Plot,embeddings
0,1975,The Candy Tangerine Man,American,Matt Cimber,John Daniels Eli Haines Tom Hankason,action,A successful Los Angeles-based businessperson ...,"[-0.06835173815488815, -0.01313861645758152, -..."
1,1975,Capone,American,Steve Carver,Ben Gazzara Susan Blakely John Cassavetes Sylv...,crime drama,The story is of the rise and fall of the Chica...,"[-0.014117980375885963, 0.0407051146030426, -0..."
2,1975,Cleopatra Jones and the Casino of Gold,American,Charles Bail,Tamara Dobson Stella Stevens,action,The story begins with two government agents Ma...,"[-0.09258949756622314, 0.011885089799761772, -..."
3,1975,Conduct Unbecoming,American,Michael Anderson,Stacy Keach Richard Attenborough Christopher P...,drama,Around 1880 two young British officers arrive ...,"[-0.07435084134340286, -0.06386178731918335, 0..."
4,1975,Cooley High,American,Michael Schultz,Lawrence Hilton-Jacobs Glynn Turman Garrett Mo...,comedy,Set in 1964 Chicago Preach an aspiring playwri...,"[-0.041632335633039474, 0.0379236564040184, -0..."


## 5. Run Filtered Similarity Searches on our KDB.AI Vector Database

#### Set up embedding model to embed our natural language queries

In [ ]:
# embedding model to be used to embed user input query
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

#### Create a query vector by using the embedding model to embed a natural language query

In [19]:
# Embed a query
query_vector = {'flat_index' : [embedding_model.encode('star wars Luke Skywalker').tolist()]}

#### Run vector similarity search, return the top-3 similar movies

In [20]:
# Search vector db to find most relevant movies
print(table.search(vectors=query_vector, n=3))

[   __nn_distance  ReleaseYear  \
0       0.748475         1983   
1       0.910225         1977   
2       0.942763         1980   

                                              Title    Origin  \
0                                Return of the Jedi  American   
1  Star Wars Episode IV: A New Hope (aka Star Wars)  American   
2                           The Empire Strikes Back  American   

           Director                                               Cast  \
0  Richard Marquand  Mark Hamill Harrison Ford Carrie Fisher Billy ...   
1      George Lucas  Mark Hamill Harrison Ford Carrie Fisher Alec G...   
2    Irvin Kershner  Carrie Fisher Harrison Ford Mark Hamill Billy ...   

             Genre                                               Plot  \
0  science fiction  Luke Skywalker initiates a plan to rescue Han ...   
1  science fiction  The galaxy is in the midst of a civil war. Spi...   
2  science fiction  Three years after the destruction of the Death...   

               

#### Repeat the search with metadata filters to narrow the search space

In [21]:
print(table.search(vectors=query_vector, n=3, filter=[("like", "Director", "George Lucas"),("=", "ReleaseYear", 1977)]))

[   __nn_distance  ReleaseYear  \
0       0.910225         1977   

                                              Title    Origin      Director  \
0  Star Wars Episode IV: A New Hope (aka Star Wars)  American  George Lucas   

                                                Cast            Genre  \
0  Mark Hamill Harrison Ford Carrie Fisher Alec G...  science fiction   

                                                Plot  \
0  The galaxy is in the midst of a civil war. Spi...   

                                          embeddings  
0  [-0.10030582547187805, 0.008335104212164879, 0...  ]


### Fuzzy Filtering
What if there are some spelling mistakes?

In [22]:
# Fuzzy filter with a misspelled name. This defaults to 'Levenshtein' distance metric
print(table.search(vectors=query_vector, n=3, filter=[['fuzzy','Director',[["Goerge Lucas",2]]]]))

[   __nn_distance  ReleaseYear  \
0       0.910225         1977   
1       1.166750         2005   
2       1.210034         2002   

                                              Title    Origin      Director  \
0  Star Wars Episode IV: A New Hope (aka Star Wars)  American  George Lucas   
1      Star Wars: Episode III   Revenge of the Sith  American  George Lucas   
2      Star Wars: Episode II   Attack of the Clones  American  George Lucas   

                                                Cast                   Genre  \
0  Mark Hamill Harrison Ford Carrie Fisher Alec G...         science fiction   
1  Ewan McGregor Hayden Christensen Natalie Portm...  science fiction action   
2   Ewan McGregor Natalie Portman Hayden Christensen         science fiction   

                                                Plot  \
0  The galaxy is in the midst of a civil war. Spi...   
1  Three years after the Battle of Geonosis the g...   
2  Ten years after the Trade Federations invasion...   

   

In [23]:
# Fuzzy filter with a misspelled genre, choosing the distance metric algorithm to use, in this case 'osa', Optimal String Alignment.
print(table.search(vectors=query_vector, n=3, filter=[['fuzzy','Genre',[["sceince fictoin",2,'osa']]]]))

[   __nn_distance  ReleaseYear  \
0       0.748475         1983   
1       0.910225         1977   
2       0.942763         1980   

                                              Title    Origin  \
0                                Return of the Jedi  American   
1  Star Wars Episode IV: A New Hope (aka Star Wars)  American   
2                           The Empire Strikes Back  American   

           Director                                               Cast  \
0  Richard Marquand  Mark Hamill Harrison Ford Carrie Fisher Billy ...   
1      George Lucas  Mark Hamill Harrison Ford Carrie Fisher Alec G...   
2    Irvin Kershner  Carrie Fisher Harrison Ford Mark Hamill Billy ...   

             Genre                                               Plot  \
0  science fiction  Luke Skywalker initiates a plan to rescue Han ...   
1  science fiction  The galaxy is in the midst of a civil war. Spi...   
2  science fiction  Three years after the destruction of the Death...   

               

#### More Examples

In [24]:
# Another query
query_vector = {'flat_index' :[embedding_model.encode('conspiracy theories involving art').tolist()]}

In [25]:
# Another filtered search example
print(table.search(vectors=query_vector, n=3, filter=[("like", "Genre", "*thriller*"),("like","Cast","*Tom Hanks*")]))

[   __nn_distance  ReleaseYear               Title    Origin          Director  \
0       1.276896         2006   The Da Vinci Code  American        Ron Howard   
1       1.395944         2017          The Circle  American    James Ponsoldt   
2       1.607655         2017            The Post  American  Steven Spielberg   

                                                Cast  \
0  Tom Hanks Audrey Tautou Ian McKellen Alfred Mo...   
1  James Ponsoldt (director/screenplay); Tom Hank...   
2  Steven Spielberg (director); Liz Hannah Josh S...   

                              Genre  \
0                          thriller   
1             sci-fi drama thriller   
2  biography drama history thriller   

                                                Plot  \
0  Jacques Sauni¨re the Louvres curator is pursue...   
1  When her car breaks down Mae Holland contacts ...   
2  In 1966 Vietnam State Department military anal...   

                                          embeddings  
0  [-0.11887

In [26]:
# Another query
query_vector = {'flat_index' :[embedding_model.encode('middle earth fantasy adventure in the Shire').tolist()]}

In [27]:
# Another filtered search example
print(table.search(vectors=query_vector, n=3, filter=[("within","ReleaseYear",[2000,2010])]))


[   __nn_distance  ReleaseYear  \
0       1.014505         2001   
1       1.099138         2002   
2       1.153412         2003   

                                               Title    Origin       Director  \
0  The Lord of the Rings: The Fellowship of the Ring  American  Peter Jackson   
1              The Lord of the Rings: The Two Towers  American  Peter Jackson   
2      The Lord of the Rings: The Return of the King  American  Peter Jackson   

                                                Cast              Genre  \
0  Elijah Wood Ian McKellen Liv Tyler Sean Astin ...            fantasy   
1  Elijah Wood Ian McKellen Liv Tyler Viggo Morte...  adventure fantasy   
2  Elijah Wood Ian McKellen Liv Tyler Sean Astin ...  adventure fantasy   

                                                Plot  \
0  In the Second Age of Middle-earth the lords of...   
1  After awakening from a dream of Gandalf the Gr...   
2  Many years ago two Hobbits Smeagol and Dagol a...   

               

#### Another Fuzzy Filtering Example on the Genre metadata column

In [28]:
# Another filtered search example, with fuzzy filtering the Director column reconciling for a typo in what should be 'Peter Jackson'
print(table.search(vectors=query_vector, n=3, filter=[("within","ReleaseYear",[2000,2010]),['fuzzy','Director',[["Peter Jacksen",1,"damerau_levenshtein"]]]]))


[   __nn_distance  ReleaseYear  \
0       1.014505         2001   
1       1.099138         2002   
2       1.153412         2003   

                                               Title    Origin       Director  \
0  The Lord of the Rings: The Fellowship of the Ring  American  Peter Jackson   
1              The Lord of the Rings: The Two Towers  American  Peter Jackson   
2      The Lord of the Rings: The Return of the King  American  Peter Jackson   

                                                Cast              Genre  \
0  Elijah Wood Ian McKellen Liv Tyler Sean Astin ...            fantasy   
1  Elijah Wood Ian McKellen Liv Tyler Viggo Morte...  adventure fantasy   
2  Elijah Wood Ian McKellen Liv Tyler Sean Astin ...  adventure fantasy   

                                                Plot  \
0  In the Second Age of Middle-earth the lords of...   
1  After awakening from a dream of Gandalf the Gr...   
2  Many years ago two Hobbits Smeagol and Dagol a...   

               

## Delete the KDB.AI Table
Once finished with the table, it is best practice to drop it.

In [29]:
table.drop()

#### Take Our Survey
We hope you found this sample helpful! Your feedback is important to us, and we would appreciate it if you could take a moment to fill out our brief survey. Your input helps us improve our content.

Take the [Survey](https://delighted.com/t/Jvlbdqvt)
